In [1]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from pclib.nn.models import SmallLinearClassifier, PCLeNetV1, PCLeNetV2
from pclib.optim.train import train
from customdataset import PreloadedDataset

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
VAL_RATIO = 0.1
# transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),                                
# ])
transform = transforms.ToTensor()

dataset = datasets.MNIST('../Datasets/', train=True, download=False, transform=transforms.ToTensor())
# shorten dataset
# length = 100
# dataset = torch.utils.data.Subset(dataset, range(length))
val_len = int(len(dataset) * VAL_RATIO)
train_len = len(dataset) - val_len
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, val_len])
train_dataset = PreloadedDataset.from_dataset(train_dataset, transform, device)
val_dataset = PreloadedDataset.from_dataset(val_dataset, transforms.ToTensor(), device)
INPUT_SHAPE = (1, 28, 28)
NUM_CLASSES = 10

In [4]:
seed = 42
torch.manual_seed(seed)

model_name = "mnist/pclenet_reduceR"
LEARNING_RATE = 3e-3
# model = SmallLinearClassifier(784, NUM_CLASSES, [256], bias=False, device=device)
model = PCLeNetV2(INPUT_SHAPE, NUM_CLASSES).to(device)
optimiser = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = torch.nn.CrossEntropyLoss()
step = 0

In [5]:
NUM_EPOCHS = 100
step = train(
    model, 
    train_dataset, 
    val_dataset, 
    optimiser, 
    criterion,
    model_name, 
    NUM_EPOCHS, 
    LEARNING_RATE, 
    flatten=False, 
    step=step, 
    device=device,
    batch_size=512,
)

Layer 0, R shape: torch.Size([512, 6, 14, 14]), E shape: torch.Size([512, 1, 28, 28])
build pred


ValueError: ConvTranspose2D: for 4D input, output_size must have 2 or 4 elements (got 3)

In [8]:
# print mean and std of weights
for layer in model.layers:
    print(f"layer.weight.mean(): {layer.weight.mean()}")
    print(f"layer.weight.std(): {layer.weight.std()}")

layer.weight.mean(): -0.007019272539764643
layer.weight.std(): 0.032445985823869705
layer.weight.mean(): -0.0017405092949047685
layer.weight.std(): 0.025935158133506775
layer.weight.mean(): -0.0027031730860471725
layer.weight.std(): 0.026070384308695793
layer.weight.mean(): 0.018842658028006554
layer.weight.std(): 0.04142702370882034
